In [1]:
from data.dataset import NERDataset
from models.networks import GlobalContextualDeepTransition
from tqdm.notebook import tqdm
import torch

In [2]:
sourceName = 'data/conll03/eng.train.src'
targetName = 'data/conll03/eng.train.trg'
data = NERDataset(sourceName, targetName)
data.numTags = 256
loader = data.getLoader(batch_size=283)

In [3]:
max(data.wordIdx.values()), len(data.wordIdx)

(23624, 23625)

In [4]:
numChars = 100
charEmbedding = 128
numWords = 23625
wordEmbedding = 300
contextOutputUnits = 128
contextTransitionNumber = transitionNumber = 5
encoderUnits = 256
decoderUnits = 256

In [5]:
model = GlobalContextualDeepTransition(numChars, charEmbedding, numWords,
                     wordEmbedding, contextOutputUnits, contextTransitionNumber,
                        encoderUnits, decoderUnits, transitionNumber)
numParams = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {numParams:,}")

Trainable parameters: 7,440,256


In [6]:
model = model.cuda()
tot = 0
for batch in tqdm(loader):
    words, chars, mask, targets = batch
    batch = words.cuda(), chars.cuda(), mask.cuda(), targets.cuda()
    loss = model.training_step(batch, 0)
    tot += loss.item()
print(tot)


115142997.43122149


In [7]:
lens = torch.Tensor([len(s) for s in data.sentences])
{
    'avg': lens.mean(),
    'std': lens.std(),
    'max': lens.max(),
    'min': lens.min()
}

{'avg': tensor(14.5019),
 'std': tensor(11.6028),
 'max': tensor(113.),
 'min': tensor(1.)}

In [8]:
4096/lens.mean()

tensor(282.4460)